In [2]:
# Simple Thin Web Server that identifies images containing apples using IBM Watson API

# Tornado is a Python library for a simple webserver
# install with pip install tornado
import tornado.web

# Watson Python library
# install with pip install watson_developer_cloud
# from watson_developer_cloud import VisualRecognitionV3

# Native modules
import os
import uuid

In [ ]:
# The code that runs when a POST request is received to /upload
class UploadHandler(tornado.web.RequestHandler) :
    def post(self):
        # Receive a file from the client
        fileinfo = self.request.files["file"][0]
        file_name = fileinfo['filename']
        extn = os.path.splitext(file_name)[1]

        # Temporarily store the file server-side
        uid = str(uuid.uuid4())
        cname = uid + extn
        saved_file = cname
        fh = open(saved_file, 'wb')
        fh.write(fileinfo['body'])
        fh.close()

        # Upload the temporary file to Watson's Image API and then delete it
        with open(cname, 'rb') as f:
            watson_result = api.classify(images_file=f)
            classes = [x["class"] for x in watson_result["images"][0]["classifiers"][0]["classes"]]
            class_string = ', '.join(classes)

        # Compose and return a simple HTML page based on the result, focusing on "Is apple / not an apple"
        result_page = ""
        if 'apple' in class_string:
            result_page += "<h1>Yep. That looks like an apple.</h1>"
        else:
            result_page += "<h1>That doesn't look like an apple.</h1>"
        result_page += "<h2>It also looks like...</h2><ul>"
        for c in classes:
            result_page += "<li>%s</li>" % c
        self.write(result_page)

In [ ]:
# The code that runs and returns a simple upload form when a GET request is received to /
class IndexHandler(tornado.web.RequestHandler) :
    def get(self):
        self.write(
            """
            <html>
               <body>
                  <h1>Is this an apple?</h1>
                  <h2>For when you're just not certain... (powered by IBM Watson).</h2>
                  <form action="upload" method="POST"
                     enctype="multipart/form-data">
                     <input type="file" name="file"/>
                     <input type="submit"/>
                  </form>
               </body>
            </html>
            """
        )

In [ ]:
settings = dict(
    debug = True
)

application = tornado.web.Application([
    (r"/upload", UploadHandler),
    (r"/", IndexHandler)
])

if __name__ == "__main__":
    port = 9999
    application.listen(port)
    api = VisualRecognitionV3(version='2016-05-20', api_key='---')
    print "Server ready on %s..." % port
    tornado.ioloop.IOLoop.instance().start()